In [ ]:
%matplotlib notebook

# This examples shows who to download files from the ONC server
import glob
import os

import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import strawb
import strawb.sensors.module
import strawb.tools

import plotly.express as px
import pandas
import plotly.io as pio
pio.templates.default = "plotly_white"

import random

plt.rcParams.update({"text.usetex": False})  # fix time fmt of x_ticks

In [ ]:
if True: # takes one file randomly, takes some seconds
    file_list = module.FileHandler.find_files('*-SDAQ-MODULE.hdf5',
                                               directory=strawb.Config.raw_data_dir,
                                               recursive=True,
                                               raise_nothing_found=True)

    full_path = random.choice(file_list)  # select a random file
else:  # faster
    full_path = '/dss/strawb/raw_module_data/tummuontracker001/old_files/sdaq/TUMMUONTRACKER001_20210806T000000.000Z_20210806T235959.999Z-SDAQ-MODULE.hdf5'
file_name = os.path.split(full_path)[-1]

In [ ]:
module = strawb.sensors.module.FileHandler(full_path)
module.file_attributes  # show the file attributes, time in seconds since epoch (1.1.1970) UTC

# Plotting with pandas and plotly

In [ ]:
df_accel = pandas.DataFrame(dict(time = module.accel_time.asdatetime()[:],
                                 x    = module.accel_x,
                                 y    = module.accel_y,
                                 z    = module.accel_z,))

In [ ]:
df_magneto = pandas.DataFrame(dict(time = module.magneto_time.asdatetime()[:],
                                   x    = module.magneto_x,
                                   y    = module.magneto_y,
                                   z    = module.magneto_z,))

In [ ]:
df_pth = pandas.DataFrame(dict(time        = module.pth_time.asdatetime()[:],
                               humidity    = module.pth_humidity,
                               temperature = module.pth_temperature,
                               pressure    = module.pth_pressure,))

In [ ]:
df_temperatures = pandas.DataFrame(dict(time  = module.temperatures_time.asdatetime()[:],
                                        temp1 = module.temperatures_temp1,
                                        temp2 = module.temperatures_temp2,
                                        temp3 = module.temperatures_temp3))

In [ ]:
df_pwrmoni = pandas.DataFrame(dict(time           = module.pwrmoni_time.asdatetime()[:],
                                   c2_current     = module.pwrmoni_c2_current,
                                   c2_voltage     = module.pwrmoni_c2_voltage,
                                   laser_current  = module.pwrmoni_laser_current,
                                   laser_voltage  = module.pwrmoni_laser_voltage,
                                   motor_current  = module.pwrmoni_motor_current,
                                   motor_voltage  = module.pwrmoni_motor_voltage,
                                   padiwa_current = module.pwrmoni_padiwa_current,
                                   padiwa_voltage = module.pwrmoni_padiwa_voltage,
                                   switch_current = module.pwrmoni_switch_current,
                                   switch_voltage = module.pwrmoni_switch_voltage,
                                   trb3sc_current = module.pwrmoni_trb3sc_current,
                                   trb3sc_voltage = module.pwrmoni_trb3sc_voltage))

In [ ]:
px.line(df_accel, x='time', y=['x', 'y', 'z'])

In [ ]:
px.line(df_magneto, x='time', y=['x', 'y', 'z'])

In [ ]:
px.line(df_pth, x='time', y=['humidity','temperature','pressure'])

In [ ]:
px.line(df_temperatures, x='time', y=['temp1','temp2','temp3'])

In [ ]:
px.line(df_pwrmoni, x='time', y=['c2_current', 'c2_voltage',
                                 'laser_current', 'laser_voltage',
                                 'motor_current', 'motor_voltage',
                                 'padiwa_current', 'padiwa_voltage',
                                 'switch_current', 'switch_voltage',
                                 'trb3sc_current', 'trb3sc_voltage'])

# Load a file
## can be a single or combined aka. virtual HDF5 file

In [ ]:
fig, ax = plt.subplots(nrows=8, squeeze=False, figsize=(7,15), sharex=True)
ax = ax.flatten()

i = 0

# ACCEL
strawb.tools.plot_binned_mean(module.accel_time, module.accel_x[:]*9.81, ax=ax[i], label='accel x')
strawb.tools.plot_binned_mean(module.accel_time, module.accel_y[:]*9.81, ax=ax[i], label='accel y')
ax[i].set_ylabel('Accel [m/s**2]')
i += 1
strawb.tools.plot_binned_mean(module.accel_time, module.accel_z[:]*9.81, ax=ax[i], label='accel z')
ax[i].set_ylabel('Accel [m/s**2]')
i += 1

# Magnet
strawb.tools.plot_binned_mean(module.magneto_time, module.magneto_x, ax=ax[i], label='magneto x')
strawb.tools.plot_binned_mean(module.magneto_time, module.magneto_y, ax=ax[i], label='magneto y')
strawb.tools.plot_binned_mean(module.magneto_time, module.magneto_z, ax=ax[i], label='magneto z')
ax[i].set_ylabel('Magneto [a.u.]')
i += 1

# TEMP.
strawb.tools.plot_binned_mean(module.temperatures_time, module.temperatures_temp1, ax=ax[i], label='temp 1')
strawb.tools.plot_binned_mean(module.temperatures_time, module.temperatures_temp2, ax=ax[i], label='temp 2')
ax[i].set_ylabel('Temp [C]')
i += 1
strawb.tools.plot_binned_mean(module.temperatures_time, module.temperatures_temp3, ax=ax[i], label='temp 3')
ax[i].set_ylabel('Temp [C]')
i += 1

# PTH
strawb.tools.plot_binned_mean(module.pth_time, module.pth_temperature, ax=ax[i], label='temp pth')
ax[i].set_ylabel('Temp [C]')
i += 1

strawb.tools.plot_binned_mean(module.pth_time, module.pth_humidity, ax=ax[i], label='pth humidity')
ax[i].set_ylabel('humidity [per]')
i += 1

strawb.tools.plot_binned_mean(module.pth_time, module.pth_pressure, ax=ax[i], label='pth pressure')
ax[i].set_ylabel('pressure [mbar]')
i += 1

for ax_i in ax:
    ax_i.legend(loc=1)
    ax_i.grid()

# # set limits
# limits = np.array([[ax_i.dataLim.x0, ax_i.dataLim.x1] for ax_i in ax])
# ax[-1].set_xlim((limits[:,0].min(), limits[:,1].max()))
    
fig.autofmt_xdate()
locator = mdates.AutoDateLocator()
#fmt = mdates.AutoDateFormatter(locator, usetex=True)
fmt = mdates.ConciseDateFormatter(locator)
ax[-1].xaxis.set_major_formatter(fmt)
plt.tight_layout()

In [ ]:
limits = np.array([[ax_i.dataLim.x0, ax_i.dataLim.x1] for ax_i in ax])
limits

## plot multiple modules (currently doesn't work: needs viratul hdf5 files or a file per moduel)

In [ ]:
# module_list = []

# for dev_i in strawb.dev_codes_deployed:
#     # get a list with all filenames that should be included in the virtual HDF5 file
#     f_names = glob.glob(f'{strawb.Config.raw_data_dir}/{dev_i}_*MODULE.hdf5')
#     f_names.sort()  # f_names should be sorted

#     # create the virtual HDF5 file, f_names should be sorted, otherwise the data isn't sorted
#     virtual_hdf5_gen = strawb.VirtualHDF5(f'{dev_i}_MODULE.h5',
#                                           f_names,
#                                           obj_dict_filter=strawb.DatasetsInGroupSameSize)

#     module_list.append(strawb.sensors.module.FileHandler(f'{dev_i}_MODULE.h5'))

In [ ]:
# fig, ax = plt.subplots(nrows=6, squeeze=False, figsize=(10,15), sharex=True)
# ax = ax.flatten()


# for module in module_list:
#     print(module.module)
#     i = 0
#     # TEMP.
#     mask = module.temperatures_time[:] > datetime.datetime(year=2020,month=12,day=1).timestamp()
#     plot_time = module.temperatures_time[:][mask]
#     strawb.tools.plot_binned_mean(plot_time, module.temperatures_temp1[:][mask],
#                                   ax=ax[i], label=f'temp 1 {module.module}')
#     ax[i].set_ylabel('Temp [C]')
#     i += 1

#     strawb.tools.plot_binned_mean(plot_time, module.temperatures_temp2[:][mask],
#                                   ax=ax[i], label=f'temp 2 {module.module}')
#     ax[i].set_ylabel('Temp [C]')
#     i += 1

#     strawb.tools.plot_binned_mean(plot_time, module.temperatures_temp3[:][mask],
#                                   ax=ax[i], label=f'temp 3 {module.module}')
#     ax[i].set_ylabel('Temp [C]')
#     i += 1

#     # PTH
#     mask = module.pth_time[:] > datetime.datetime(year=2020,month=12,day=1).timestamp()
#     plot_time = module.pth_time[:][mask]
#     strawb.tools.plot_binned_mean(plot_time, module.pth_temperature[:][mask],
#                                   ax=ax[i], label=f'temp pth {module.module}')
#     ax[i].set_ylabel('Temp [C]')
#     i += 1

#     strawb.tools.plot_binned_mean(plot_time, module.pth_humidity[:][mask],
#                                   ax=ax[i], label=f'pth humidity {module.module}')
#     ax[i].set_ylabel('humidity [per]')
#     i += 1

#     strawb.tools.plot_binned_mean(plot_time, module.pth_pressure[:][mask],
#                                   ax=ax[i], label=f'pth pressure {module.module}')
#     ax[i].set_ylabel('pressure [mbar]')
#     i += 1

# for ax_i in ax:
#     ax_i.legend(loc=0)
#     ax_i.grid()

# # set limits
# limits = np.array([[ax_i.dataLim.x0, ax_i.dataLim.x1] for ax_i in ax])
# ax[-1].set_xlim((limits[:,0].min(), limits[:,1].max()))
    
# fig.autofmt_xdate()
# locator = mdates.AutoDateLocator()
# #fmt = mdates.AutoDateFormatter(locator, usetex=True)
# fmt = mdates.ConciseDateFormatter(locator)
# ax[-1].xaxis.set_major_formatter(fmt)
# plt.tight_layout()